<a href="https://colab.research.google.com/github/jmanning21/31005_MachineLearning_Python/blob/master/Machine_Learning_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Machine Learning Project
####Affnan Amir (13528841) and Julia Manning (12875795)

###Introduction
The algorithm chosen for this project is the ID3 Decision Tree building algorithm. In using this alrogithm, the data  

* Input = titanic dataset from given url:
https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
* Output = predicted resutls for those individuals which survived (i.e. binary options; Did they survive? Yes or No).

###Exploration

Challenges:
 - over-fitting the data

###Methodology

In [0]:
# Importing the required packages
import os
import numpy as np
import pandas as pd
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [0]:
# Importing the dataset from a url
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

# Read the csv and convert to a dataframe
d = pd.read_csv(url)
df = pd.DataFrame(data=d)

# See the first six rows of the dataset
df.head(6)

In [0]:
# Cleaning some of the attributes: replace null values with median values
# Replacing NA values in age with the median age of each sex
median_values = df.groupby('Sex')['Age'].transform('median')
df['Age'].fillna(median_values, inplace=True)
df.head(6) # see the first 6 rows to see if the NA value was replaced in Ln 6

In [0]:
# Check the information about the dataset
df.info()
# There are no missing values in the dataset

In [0]:
# Identify the target variable
df['Survived'],class_names = pd.factorize(df['Survived'])

print(class_names)
print(df['Survived'].unique())

# The algorithm requires the target variable to be in integer format, which it already is. 
# Therefore no data type changes need to be made.

In [0]:
# Adding and removing specific columns which may help with the accuracy of the accuracy.
# Add a Child.Adult column based on the age of the individual
df['Child.Adult'] = 'Adult'

# Child = Age < 13 and Adult = Age >= 13 
df['Child.Adult'][df['Age'] < 13.0] = 'Child'

# Remove 'Name' column as it does not provide any relevant information to the dataset, as all the values are unique
del df['Name']
# Remove 'Cabin' as there are too many NA values and the remaining values are all unique, make it impossible to determine a suitable replacement for the NA values
del df['Cabin']

In [0]:
# Identifying the predictor variables and encode any string variables to their equivalent integer formats
df['Sex'],_ = pd.factorize(df['Sex'])
df['Embarked'],_ = pd.factorize(df['Embarked'])
df['Child.Adult'],_ = pd.factorize(df['Child.Adult'])
df['Ticket'],_ = pd.factorize(df['Ticket'])

df.head(10)

In [0]:
# Re-ordering the columns to put the 'Survived' column last
df = df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked','Child.Adult','Survived']]
df.head(6)

In [0]:
# Selecting the predictor feature and target variable
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [0]:
# split data randomly into 60% training and 40% test
train_data_split = 0.6 #@param {type:"number", min:0.01, max:0.99}
test_data_split = 1 - train_data_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_data_split, random_state=0)

In [0]:
# training/model fitting
# train the decision tree
dtree = tree.DecisionTreeClassifier(criterion = 'gini', max_depth = 5, random_state = 0)
dtree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [0]:
# use model to make predictions with 
y_pred = dtree.predict(X_test)

# see how good the model performs
misclassified_count = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(misclassified_count))
accuracy = metrics.accuracy_score(y_test,y_pred)
print('Accuracy: {:.4f}'.format(accuracy))

Misclassified samples: 77
Accuracy: 0.7843


###Alternate way

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [0]:
# Function importing dataset and cleaning
def importdata(): 
  url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
  # Read the csv and convert to a dataframe
  d = pd.read_csv(url)
  df = pd.DataFrame(data=d)

  # See the first six rows of the dataset
  df.head(6)

  # Adding and removing specific columns which may help with the accuracy of the accuracy.
  # Add a Child.Adult column based on the age of the individual
  df['Child.Adult'] = 'Adult'

  # Child = Age < 13 and Adult = Age >= 13 
  df['Child.Adult'][df['Age'] < 13.0] = 'Child'

  # Remove 'Name' column as it does not provide any relevant information to the dataset, as all the values are unique
  del df['Name']
  # Remove 'Cabin' as there are too many NA values and the remaining values are all unique, make it impossible to determine a suitable replacement for the NA values
  del df['Cabin']

  # Re-ordering the columns to put the 'Survived' column last
  df = df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked','Child.Adult','Survived']]
  
  df['Sex'],_ = pd.factorize(df['Sex'])
  df['Embarked'],_ = pd.factorize(df['Embarked'])
  df['Child.Adult'],_ = pd.factorize(df['Child.Adult'])
  df['Ticket'],_ = pd.factorize(df['Ticket'])
  
  return df

In [0]:
# function to split the data
def splitdataset(df,split):
  # Seperating the target variable 
    X = df.iloc[:,:-1]
    Y = df.iloc[:,-1] 
    test_split = 1 - split
    
    # Spliting the dataset into train and test 
    X_train, X_test, y_train, y_test = train_test_split(  
    X, Y, test_size = test_split, random_state = 100) 
    
    return X, Y, X_train, X_test, y_train, y_test

In [0]:
# Function to perform training with giniIndex. 
def train_using_gini(X_train, X_test, y_train): 
  
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", 
            random_state = 100,max_depth=3, min_samples_leaf=5) 
  
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 

In [0]:
# Function to perform training with entropy. 
def train_using_entropy(X_train, X_test, y_train): 
  
    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 100, 
            max_depth = 3, min_samples_leaf = 5) 
  
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy

In [0]:
# Function to make predictions 
def prediction(X_test, clf_object): 
  
    # Predicton on test with giniIndex 
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:") 
    print(y_pred) 
    return y_pred

In [0]:
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: ", 
        confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
      
    print("Report : ", 
    classification_report(y_test, y_pred))

In [0]:
# Driver code 
def main(): 
      
    # Building Phase 
    data = importdata()
    X, Y, X_train, X_test, y_train, y_test = splitdataset(data,var) 
    clf_gini = train_using_gini(X_train, X_test, y_train) 
    clf_entropy = train_using_entropy(X_train, X_test, y_train) 
      
    # Operational Phase 
    print("Results Using Gini Index:") 
      
    # Prediction using gini 
    y_pred_gini = prediction(X_test, clf_gini) 
    cal_accuracy(y_test, y_pred_gini) 
      
    print("Results Using Entropy:") 
    # Prediction using entropy 
    y_pred_entropy = prediction(X_test, clf_entropy) 
    cal_accuracy(y_test, y_pred_entropy) 

In [77]:
# Run the entire code
var = float(input("Enter the training data split number: "))
if type(var) is float:
  main()
else:
  print("")
  print("Enter a number between 0.01 and 0.99")
  var = input("Enter the training data split number: ")

Enter the training data split number: 0.6


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


ValueError: ignored

###Evaluation
Improvements for future renditions: 
- When producing the ages for null values in the dataset, the ages could have been grouped by class and gender of the individual, rather than just using gender. This may produce a more accurate number. The age range for females is: 0.75-63, giving a range difference of 62.25, whilst the age range for males is: 0.42-80, giving a age range of 79.58. The age range is also specified further when Pclass is added.
- Use Gini Idex instead of Entropy?

###Results
This is the results from the decision tree model, displaying the accuracy outcomes for the changing variables

Train Data Split|Criterion|Max Depth|Misclassified|Accuracy
---|---|---|---|---
0.6|Entropy|3|70|0.8039
0.6|Entropy|5|68|0.8095
0.6|Gini|3|67|0.8123
0.6|Gini|5|77|0.7843
0.7|Entropy|3|48|0.8209
0.7|Entropy|5|48|0.8209
0.7|Gini|3|48|0.8209
0.7|Gini|5|49|0.8172

###Conclusion

###Ethical Justification

- potential misuses of the technique
- utilitarian view?

###Video Pitch
*Attached to the final document*